In [1]:
from sentence_classification import *

In [ ]:
lemmas = extract_lemmas(X)
pos_tags = load_pos_tags("data/resources/pos_tag_sent.txt")
pos_tag_feature = create_pos_tag_feature(X, pos_tags)
lemmas_pos = extract_pos_lemmas(X)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
svc = LinearSVC(class_weight='auto')
predicted = []
gold = []
f_measures = []
for i in range(1, 6):
    f_train = "data/folds/" + str(i) + "/train.txt"
    f_test = "data/folds/" + str(i) + "/test.txt"
    X_train, y_train = load_data(f_train)
    X_test, y_test = load_data(f_test)
    X_train_features = create_features(vectorizer, X_train, True)
    svc.fit(X_train_features, y_train)
    X_test_features = create_features(vectorizer, X_test, False)
    predicted_fold = svc.predict(X_test_features)
    predicted.extend(predicted_fold)
    gold.extend(y_test)
    f_measures.append(metrics.f1_score(y_test, predicted_fold, average='macro'))
print f_measures
print classification_report(gold, predicted, digits=3)
print metrics.precision_score(gold, predicted, average='macro')
print metrics.recall_score(gold, predicted, average='macro')
print metrics.f1_score(gold, predicted, average='macro')